## Tutorial 2: Using the generic interface to access model attributes

`COBREXA` makes use of a generic interface that enforces a uniform set of accessor functions for model attributes. This approach ensures that any custom function you create and test using, e.g. `JSONModel`s, also works on, e.g. `SBMLModel`s, `MATModel`s, `StandardModel`s, `CoreModel`s, and `CoreModelCoupled`. 

### Load the example models
As before, we are going to use the *E. coli* toy model to demonstrate the capabilities of `COBREXA`.

In [2]:
# Download the model files if they don't already exist
!isfile("e_coli_core.mat") && download("http://bigg.ucsd.edu/static/models/e_coli_core.mat", "e_coli_core.mat")
!isfile("e_coli_core.json") && download("http://bigg.ucsd.edu/static/models/e_coli_core.json", "e_coli_core.json")
!isfile("e_coli_core.xml") &&  download("http://bigg.ucsd.edu/static/models/e_coli_core.xml", "e_coli_core.xml")

using COBREXA

### Examples of accessor functions

Here is a partial list of the accessor functions:
1. `reactions`
2. `n_reactions`
3. `metabolites`
4. `n_metabolites`
5. `genes`
6. `n_genes`
7. `stoichiometry`
8. `bounds`
9. `balance`
10. `objective`
11. `coupling`
12. `n_coupling_constraints`
13. `coupling_bounds`
14. `reaction_gene_association`
15. `metabolite_formula`
16. `metabolite_charge`
17. `metabolite_compartment`
18. `reaction_subsystem`
19. `reaction_annotations`
20. `metabolite_annotations`
21. `gene_annotations`
22. `reaction_notes`
23. `metabolite_notes`
24. `gene_notes` 

Refer to the function documentation in the `COBREXA` docs for more information. 

The documentation of each accessor function is easily accessed using the normal Julia `?` command followed by the function name. For example:

In [2]:
?stoichiometry

search: stoichiometry



```
stoichiometry(a::MetabolicModel)::SparseMat
```

Get the sparse stoichiometry matrix of a model.

---

```
stoichiometry(a::CoreModel)::SparseMat
```

`CoreModel` stoichiometry matrix.

---

```
stoichiometry(model::JSONModel)
```

Get the stoichiometry. Assuming the information is stored in reaction object under key `.metabolites`.

---

```
stoichiometry(m::MATModel)
```

Extract the stoichiometry matrix, stored under key `S`.

---

```
stoichiometry(a::SBMLModel)::SparseMat
```

Recreate the stoichiometry matrix from the SBML model.

---

```
stoichiometry(model::StandardModel)
```

Return the stoichiometric matrix associated with `model` in sparse format.


### Using the accessor functions

The generic interface makes it easy to implement your own functions, should you desire to do so. 

For example, suppose you wanted to implement classic flux balance analysis on your own and you want to call this function `my_own_fba_function`. 

Note, you shouldn't do this because`COBREXA` supplies the function `flux_balance_analysis`. However, here we demonstrate the utility of the generic accessor interface in making functions that are robust against the underlying type of model.

In [4]:
# load all the models! 
json_model = load_model("e_coli_core.json")
mat_model = load_model("e_coli_core.mat")
sbml_model = load_model("e_coli_core.json")
std_model = load_model(StandardModel, "e_coli_core.mat")
core_model = load_model(CoreModel, "e_coli_core.mat")
# core_coupled_model = load_model(CoreModelCoupled, "e_coli_core.mat") # broken..

# load the optimization toolkit
using JuMP # Julia's state of the art optimization modeling package, this is a COBREXA dependency
using Tulip # optimization solver, if you don't have it add it with ] add Tulip

# make your own function using accessors
"""
    my_own_fba_function(model)

Perform FBA on `model` and return the biomass objective function flux.
"""
function my_own_fba_function(model)
    m, n = size(stoichiometry(model)) # accessor
    xl, xu = bounds(model) # accessor

    optimization_model = JuMP.Model(Tulip.Optimizer)
    @variable(optimization_model, x[i = 1:n])
    @objective(optimization_model, Max, objective(model)' * x) # accessor
    @constraint(optimization_model, stoichiometry(model) * x .== balance(model)) # mass balance
    @constraint(optimization_model, xl .<= x) # lower bounds
    @constraint(optimization_model, x .<= xu) # upper bounds
    optimize!(optimization_model)
    
    biomass_index = first(indexin(["BIOMASS_Ecoli_core_w_GAM"], reactions(model))) # accessor
    return value(x[biomass_index]) # will return the biomass objective function flux only
end

my_own_fba_function

In [5]:
my_own_fba_function(json_model) # returns the biomass objective function flux

0.8739215022690006

In [6]:
# The custom function now works on every model! 
my_own_fba_function(json_model) == my_own_fba_function(mat_model) == 
my_own_fba_function(sbml_model) == my_own_fba_function(std_model) == 
my_own_fba_function(core_model)

true